In [1]:
import pandas as pd

# Definir las rutas
file_path = 'input/expert_assessment.csv'
output_csv = 'output/expert_rankings.csv'

# Cargar el dataset
df = pd.read_csv(file_path)

# Crear la columna 'story_id' combinando 'title_id' y 'synopsis_writer'
df['story_id'] = df['title_id'] + '_' + df['synopsis_writer']

# Definir las columnas de puntuación a considerar
score_columns = [
    '1_attractive_style', '1_attractive_theme', '2_originality_style', 
    '2_originality_theme', '4_creativity_synopsis', '6_anthology', '6_own_voice'
]

# Calcular la suma total de puntuación para cada historia
df['total_score'] = df[score_columns].sum(axis=1)

def assign_ranking(group):
    group = group.copy()
    # Ordenar las historias por puntuación total de forma descendente
    group = group.sort_values('total_score', ascending=False)
    # Asignar ranking: en caso de empate se asigna la misma posición (dense ranking)
    group['ranking_pos'] = group['total_score'].rank(method='dense', ascending=False).astype(int)
    # El valor del ranking es la puntuación total (para conservar la magnitud)
    group['ranking_value'] = group['total_score']
    # Calcular el ranking value normalizado por usuario (min-max)
    min_score = group['ranking_value'].min()
    max_score = group['ranking_value'].max()
    if max_score - min_score != 0:
        group['ranking_value_norm'] = (group['ranking_value'] - min_score) / (max_score - min_score)
    else:
        group['ranking_value_norm'] = 0.0
    return group

# Aplicar la función de ranking para cada usuario
df_ranked = df.groupby('username').apply(assign_ranking).reset_index(drop=True)

# Seleccionar las columnas deseadas
df_output = df_ranked[['username', 'story_id', 'ranking_pos', 'ranking_value', 'ranking_value_norm']]

# Guardar el resultado en un CSV
df_output.to_csv(output_csv, index=False)

print(f"CSV file created: {output_csv}")

CSV file created: output/expert_rankings.csv


/tmp/ipykernel_420962/2577701097.py:40: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_ranked = df.groupby('username').apply(assign_ranking).reset_index(drop=True)
